# Final Presentation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

%matplotlib inline

In [2]:
plt.style.use('seaborn')
plt.rcParams["figure.figsize"] = 10, 6
plt.rcParams["figure.dpi"] = 150

In [3]:
# https://www.cftc.gov/MarketReports/CommitmentsofTraders/HistoricalCompressed/index.htm
import requests, zipfile, io
import os.path

if not os.path.exists("../data/F_Disagg06_16.txt"):
    url = "https://www.cftc.gov/files/dea/history/fut_disagg_txt_hist_2006_2016.zip"

    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall("../data")

In [4]:
# low_memory=False suppresses a mixed-type warning that can arise w/ messy data. We're not worried about performance right now, so not a big deal.
df = pd.read_csv("../data/F_Disagg06_16.txt", low_memory=False)
df.head()

,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,Report_Date_as_YYYY-MM-DD,CFTC_Contract_Market_Code,CFTC_Market_Code,CFTC_Region_Code,CFTC_Commodity_Code,Open_Interest_All,Prod_Merc_Positions_Long_All,Prod_Merc_Positions_Short_All,...,Conc_Net_LE_4_TDR_Long_Other,Conc_Net_LE_4_TDR_Short_Other,Conc_Net_LE_8_TDR_Long_Other,Conc_Net_LE_8_TDR_Short_Other,Contract_Units,CFTC_Contract_Market_Code_Quotes,CFTC_Market_Code_Quotes,CFTC_Commodity_Code_Quotes,CFTC_SubGroup_Code,FutOnly_or_Combined
0,WHEAT-SRW - CHICAGO BOARD OF TRADE,161227,2016-12-27,001602,CBT,0,1,446777,77925,75382,...,23.7,18.5,35.4,29.0,"(CONTRACTS OF 5,000 BUSHELS)",001602,CBT,1,A10,FutOnly
1,WHEAT-SRW - CHICAGO BOARD OF TRADE,161220,2016-12-20,001602,CBT,0,1,447386,76390,70886,...,23.9,17.4,35.1,28.0,"(CONTRACTS OF 5,000 BUSHELS)",001602,CBT,1,A10,FutOnly
2,WHEAT-SRW - CHICAGO BOARD OF TRADE,161213,2016-12-13,001602,CBT,0,1,443306,72600,76785,...,24.6,17.3,34.8,27.6,"(CONTRACTS OF 5,000 BUSHELS)",001602,CBT,1,A10,FutOnly
3,WHEAT-SRW - CHICAGO BOARD OF TRADE,161206,2016-12-06,001602,CBT,0,1,436214,77532,61394,...,26.8,17.9,38.8,28.9,"(CONTRACTS OF 5,000 BUSHELS)",001602,CBT,1,A10,FutOnly
4,WHEAT-SRW - CHICAGO BOARD OF TRADE,161129,2016-11-29,001602,CBT,0,1,444341,77239,63334,...,29.1,16.2,39.5,27.3,"(CONTRACTS OF 5,000 BUSHELS)",001602,CBT,1,A10,FutOnly


In [8]:
df_interesting = df[['Market_and_Exchange_Names',
                     'Report_Date_as_YYYY-MM-DD',
                     'CFTC_SubGroup_Code', 
                     'Prod_Merc_Positions_Long_All',
                     'Swap_Positions_Long_All',
                     'M_Money_Positions_Long_All'
                    ]]
df_interesting = df_interesting.rename(columns={
                               "Market_and_Exchange_Names": "Market and Exchange", 
                               "Report_Date_as_YYYY-MM-DD": "Date", 
                               "CFTC_SubGroup_Code": "Subgroup Code", 
                               "Prod_Merc_Positions_Long_All": "Producer/Merchant Positions",
                               "Swap_Positions_Long_All": "Swap Positions",
                               "M_Money_Positions_Long_All": "Money Manager Positions"
                              })
df_interesting.head()

,Market and Exchange,Date,Subgroup Code,Producer/Merchant Positions,Swap Positions,Money Manager Positions
0,WHEAT-SRW - CHICAGO BOARD OF TRADE,2016-12-27,A10,77925,77810,77028
1,WHEAT-SRW - CHICAGO BOARD OF TRADE,2016-12-20,A10,76390,79014,79123
2,WHEAT-SRW - CHICAGO BOARD OF TRADE,2016-12-13,A10,72600,76784,85011
3,WHEAT-SRW - CHICAGO BOARD OF TRADE,2016-12-06,A10,77532,78071,75214
4,WHEAT-SRW - CHICAGO BOARD OF TRADE,2016-11-29,A10,77239,82479,77729
